- Tachometer.
- Imbalance.
- 10g.
- 13.9264.csv
- Sample Rate: 51200

- column 1 tachometer signal that allows to estimate rotation frequency;
- columns 2 to 4 underhang bearing accelerometer (axial, radiale tangential direction);
- columns 5 to 7 overhang bearing accelerometer (axial, radiale tangential direction);
- column 8 microphone.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

- Instalar fsspec pelo terminal...

se falhar: pip install --upgrade --force-reinstall fsspec


In [ ]:
#tac_imbal_10g = '/kaggle/input/mafaulda-full/mafaulda/imbalance/10g/13.9264.csv'
tac_imbal_10g = 'C:/Files/mafaulda_full/imbalance/10g/13.9264.csv'



- pip install datawrangler

In [ ]:
df = pd.read_csv(tac_imbal_10g, names = ['f_rotac', 'under_x', 'under_y', 'under_z', 'over_x', 'over_y', 'over_z', 'micro'])
df.head(5)



In [ ]:
names = ['f_rotac', 'under_x', 'under_y', 'under_z', 'over_x', 'over_y', 'over_z', 'micro']
# fig, axes = plt.subplots(1, len(names), figsize=(len(names)*4, 4))
fig, axes = plt.subplots(2, 4, figsize=(16, 8))

axes = axes.flatten()


for i, col in enumerate(names):
    axes[i].hist(df[col], bins=30, color='blue', alpha=0.7)
    # axes[i].plot(df[col])
    # axes[i].set_title(col)
    axes[i].set_title(f'#{col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequência')

fig.suptitle('TAC Imbalance 10g', fontsize=20)

plt.tight_layout()
fig.subplots_adjust(top=0.9)
plt.show()

In [ ]:

plt.plot(df.iloc[:50, 0], color='blue', label='Tacômetro')
plt.title('50 primeiras amostras do Sinal do Tacômetro')
plt.xlabel('Amostras')
plt.ylabel('Sinal')
# Exibir a legenda e mostrar o gráfico
plt.tight_layout()
plt.show()

- Calculando a frequência do sinal e a velocidade em RPM::

Total de amostras = Taxa de amostragem × Tempo de duração.

A taxa de amostragem foi configurada para 50 kHz, ou seja, 50.000 amostras por segundo. Se temos um total de 250.000 amostras no espaço de tempo de 5 segundos:

Total de amostra / taxa de amostragem = 250000 / 50000 = 5 segundos.

Total de amostragem = 50.000 amostras x 5 segundos.

Taxa de amostragem = 50.000.

Cálculo de rpm:

RPM = Frequencia (Hz) × 60


- Gráfico de amplitude em função de amostragem:

- Esse cálculo é típico para sinais de tacômetros, onde se quer determinar a frequência de rotação (em Hz) com base nos cruzamentos pelo ponto médio.
Nesse caso, tac_signal representa o sinal do tacômetro, e crossing_times (calculado anteriormente) contém os tempos onde o sinal cruza a média. Os intervalos entre esses cruzamentos (time_time) representam os períodos de cada ciclo.

In [ ]:
tach_signal = df.iloc[:, 0]

#detecção de passagem or zero:
def zero_crossing(signal):
    #cálculo de cruzamento em zero:
    zero_cross = np.where(np.diff(np.sign(signal)))[0]
    return zero_cross

#Detecção de passagem por zero:
zero_crosses_indices = zero_crossing(tach_signal)

# Contagem de tempo entre as passagens por zero, com amostragem de 50KHz:
#Ta = 50000
Ta = 0.00002  # Tempo de amostragem (50 kHz)

time_time = np.diff(zero_crosses_indices) * Ta  # Tempo entre passagens por zero

# Cálculo do período méd. e frequência média de rotação do array time_time:
if len(time_time) > 0:  # Verifica se há intervalos
    periodo_medio = np.mean(time_time)
    freq_rotacao = 1 / periodo_medio if periodo_medio != 0 else 0  # Evita divisão por zero
else:
    periodo_medio = 0
    freq_rotacao = 0



print(f'Frequência média de rotação: {frequencia_media:.4f} Hz')
print(f'Período médio: {periodo_medio:.4f} s')





In [ ]:

# Converter tach_signal para array NumPy
tach_signal = np.array(tach_signal)

#criar o eixo do tempo com o mesmo tamanho do sinal do tacômetro 'tach_signal'. Assumindo que o sinal foi amostrado a 50 kHz (Ta = 0.00002 s):
# Ta = 0.00002  # Tempo de amostragem (50 kHz)
time = np.arange(len(tach_signal)) * Ta  # Eixo do tempo (em segundos)

plt.figure(figsize=(14, 6))
plt.plot(time, tach_signal, label='Sinal do Tacômetro')
plt.scatter(time[zero_crosses_indices], tach_signal[zero_crosses_indices], color='red', label='Cruzamentos por Zero', zorder=5)

# Personalizar o gráfico
plt.title('Sinal do Tacômetro com Cruzamentos por Zero')
plt.xlabel('Tempo (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)

plt.show()

- Gráfico de amplitude em função do tempo, com identificação dos pontos de inflexão (onde a derivada do sinal muda de sinal)

In [ ]:
import scipy.signal as signal
from scipy.signal import find_peaks

# Encontrar os picos no sinal

# Criar o eixo de tempo
time = np.arange(len(tac_signal)) / Ta
peaks, _ = find_peaks(tac_signal, height=0)

#Encontrar as inversões de sinal nos vales (Mínimos):

vales, _ = find_peaks(-tac_signal, height=0)

#combinação de índices de picos e vales:
peaks_and_vales = np.concatenate((peaks, vales))
# Encontrar os cruzamentos por zeros
zero_crosses_indices = np.where(np.diff(np.sign(tac_signal)))[0]


# Gerar o gráfico
plt.figure(figsize=(10, 6))
plt.plot(time, tac_signal, label='Sinal do Tacômetro')
plt.scatter(time[zero_crosses_indices], tac_signal[zero_crosses_indices], color='red', label='Cruzamentos por Zero', zorder=5)

# Personalizar o gráfico
plt.title('Sinal do Tacômetro com Cruzamentos por Zero')
plt.xlabel('Tempo (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)

# Exibir o gráfico
plt.show()

- as linhas representam o sinal ao longo do tempo, e os pontos em vermelho indicam os cruzamentos pelo eixo zero que são as mudanças de sinal (de positivo para negativo ou vice-versa). Eles são usados para calcular a frequência de rotação. Essa oscilação reflete o comportamento do tacômetro. A faixa entre -1 e 0 faz parte dessa oscilação natural do sinal.

- Os cruzamentos estão distribuídos de maneira relativamente regular, um processo limpo, ao longo do tempo. Isso sugere que a rotação do sistema é estável, sem grandes flutuações ou variações bruscas na frequência de rotação. Essa regularidade é um indicativo de que o sistema está funcionando de forma consistente, sem mudanças repentinas na velocidade. Não foi detectado a presença de ruídos significativos.

- Vamos calcular o período e frequência médios, o menor e o maior período.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Supondo que tac_signal e Ta estão definidos
# Exemplo: tac_signal = df_10['f_rotac'], Ta = 1/fs

# Converter tac_signal para array NumPy
tac_signal = np.array(tac_signal)

# Criar o eixo de tempo
time = np.arange(len(tac_signal)) / Ta

# Calcular a média do sinal
mean_signal = np.mean(tac_signal)

# Subtrair a média para centralizar o sinal em torno de zero
tac_signal_centered = tac_signal - mean_signal

# Encontrar os cruzamentos pela linha média
zero_crosses_indices = np.where(np.diff(np.sign(tac_signal_centered)))[0]

# Ajustar índices para corresponder aos pontos exatos de cruzamento
# Filtra índices onde o sinal realmente cruza a média
crossings = []
for idx in zero_crosses_indices:
    if idx + 1 < len(tac_signal_centered):
        # Verifica se o produto dos valores adjacentes é negativo (indica cruzamento)
        if tac_signal_centered[idx] * tac_signal_centered[idx + 1] <= 0:
            # Interpolar para encontrar o ponto exato de cruzamento
            fraction = abs(tac_signal_centered[idx]) / (abs(tac_signal_centered[idx]) + abs(tac_signal_centered[idx + 1]))
            crossing_time = time[idx] + fraction * (time[idx + 1] - time[idx])
            crossing_value = mean_signal  # O valor na média
            crossings.append((crossing_time, crossing_value))

# Separar os tempos e valores dos cruzamentos
if crossings:
    crossing_times, crossing_values = zip(*crossings)
else:
    crossing_times, crossing_values = [], []

# Gerar o gráfico
plt.figure(figsize=(14, 6))
plt.plot(time, tac_signal, label='Sinal do Tacômetro')
plt.scatter(crossing_times, crossing_values, color='red', label='Cruzamentos pelo Ponto Médio', zorder=2)

# Adicionar linha horizontal na média
plt.axhline(mean_signal, color='green', linestyle='--', label='Ponto Médio (Média)')

# Personalizar o gráfico
plt.title('Sinal do Tacômetro com Cruzamentos pelo Ponto Médio')
plt.xlabel('Tempo (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)

# Exibir o gráfico
plt.show()

In [ ]:
# Converter tach_signal para array NumPy
tach_signal = np.array(tach_signal)

# Criar o eixo do tempo (Ta = 0.00002 s, 50 kHz)
Ta = 0.00002  # Tempo de amostragem (50 kHz)
time = np.arange(len(tach_signal)) * Ta  # Eixo do tempo (em segundos)

# Filtrar os dados para o intervalo de 1 a 2 segundos
mask = (time >= 1) & (time <= 2)  # Máscara para o intervalo
time_interval = time[mask]  # Tempos no intervalo
tach_signal_interval = tach_signal[mask]  # Sinal no intervalo

# Filtrar os índices de cruzamentos por zero para o intervalo
zero_crosses_interval = zero_crosses_indices[(zero_crosses_indices >= np.where(mask)[0][0]) & 
                                             (zero_crosses_indices <= np.where(mask)[0][-1])]

# Ajustar os índices para o novo intervalo
zero_crosses_interval = zero_crosses_interval - np.where(mask)[0][0]

# Gerar o gráfico
plt.figure(figsize=(14, 6))
plt.plot(time_interval, tach_signal_interval, label='Sinal do Tacômetro')
plt.scatter(time_interval[zero_crosses_interval], tach_signal_interval[zero_crosses_interval], 
            color='red', label='Cruzamentos por Zero', zorder=5)

# Personalizar o gráfico
plt.title('Sinal do Tacômetro com Cruzamentos por Zero (1 a 2 segundos)')
plt.xlabel('Tempo (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)

# Exibir o gráfico
plt.show()


In [ ]:
# Calcular a variabilidade entre os períodos (tempo entre cruzamentos zero)
time_crossings = zero_crosses_indices * Ta  # Converter os índices de cruzamento em tempo
periods = np.diff(time_crossings)  # Diferença entre os tempos dos cruzamentos, que nos dá os períodos

# Calcular métricas de variabilidade
mean_period = np.mean(periods)
std_period = np.std(periods)  # Desvio padrão dos períodos
min_period = np.min(periods)
max_period = np.max(periods)


In [ ]:
# Criando o dicionário de dados para a tabela
data = {
    'Métrica': ['Período Médio', 'Desvio Padrão do Período', 'Período Mínimo', 'Período Máximo', 'per med', 'Frequência Média'],
    'Descrição': ['Média do período entre cruzamentos de zero.', 
                  'Desvio padrão do período entre cruzamentos de zero.', 
                  'Período mínimo entre cruzamentos de zero.',
                  'Período máximo entre cruzamentos de zero.',
                  'primeiro período médio entre cruzamentos de zero.',
                  'Frequência média estimada baseada nos períodos.'],
    'Valor': [mean_period, std_period, min_period, max_period, periodo_medio, frequencia_media]
}

# Convertendo para um DataFrame
df = pd.DataFrame(data)

# Adicionando bordas à tabela
df_styled = df.style.set_table_styles(
    [{'selector': 'th', 'props': [('border', '1px solid black')]},
     {'selector': 'td', 'props': [('border', '1px solid black')]}]
)

# Exibindo a tabela com bordas
df_styled

In [ ]:
# Gerar dados de uma distribuição normal com base no período médio e desvio padrão
# x = np.linspace(min_period, max_period, 1000)
# normal_distribution = (1 / (std_period * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mean_period) / std_period) ** 2)
# x_desvio = (x - mean_period, 1000)
# x_percent = x_desvio * 100

deviation_min = (min_period - mean_period) / std_period  # Desvio mínimo em unidades de std
deviation_max = (max_period - mean_period) / std_period  # Desvio máximo em unidades de std

# Criar escala percentual (de -X% a +X% do desvio padrão)
x_deviation = np.linspace(deviation_min, deviation_max, 1000)  # Desvios em unidades de std
x_percent = x_deviation * 100  # Converter para percentual

# Calcular a distribuição normal usando x_deviation
normal_distribution = (1 / (std_period * np.sqrt(2 * np.pi))) * np.exp(-0.5 * x_deviation**2)

""" distribuicao_normal = (1 / (std_period * np.sqrt(2*np.pi))) * np.exp(-0.5 * ((x - mean_period) / std_period) ** 2)

# Criar o gráfico
plt.figure(figsize=(10, 6))
plt.plot(x, distribuicao_normal, label='Distribuição Normal dos Períodos', color='blue')

# Adicionar marcações para o período médio e desvio padrão
plt.axvline(mean_period, color='red', linestyle='--', label=f'Período Médio: {mean_period:.4f}s')
plt.axvline(mean_period + std_period, color='green', linestyle='--', label=f'+1 Desvio Padrão: {mean_period + std_period:.4f}s')
plt.axvline(mean_period - std_period, color='green', linestyle='--', label=f'-1 Desvio Padrão: {mean_period - std_period:.4f}s')

# Personalizar o gráfico
plt.title('Distribuição Normal dos Períodos de Cruzamento por Zero')
plt.xlabel('Período (s)')
plt.ylabel('Densidade de Probabilidade')
plt.legend()
plt.grid(True)

# Exibir o gráfico
plt.show()"""

# Criar o gráfico
plt.figure(figsize=(10, 6))
plt.plot(x_percent, normal_distribution, label='Distribuição Normal dos Períodos', color='blue')

# Adicionar marcações para o período médio e desvio padrão (em percentual)
plt.axvline(0, color='red', linestyle='--', label='Período Médio: 0%')  # Média está em 0%
plt.axvline(100, color='green', linestyle='--', label='+1 Desvio Padrão: +100%')  # +1 std = +100%
plt.axvline(-100, color='green', linestyle='--', label='-1 Desvio Padrão: -100%')  # -1 std = -100%

# Personalizar o gráfico
plt.title('Distribuição Normal dos Períodos (Escala Percentual de Desvio Padrão)')
plt.xlabel('Desvio Percentual do Desvio Padrão (%)')
plt.ylabel('Densidade de Probabilidade')
plt.legend()
plt.grid(True)

# Exibir o gráfico
plt.show()

#### Transformada De Fourier Do Sinal Do Tacômetro

In [ ]:
# Aplicar a Transformada de Fourier no sinal do tacômetro
tach_signal_fft = np.fft.fft(tach_signal)
frequencies = np.fft.fftfreq(len(tach_signal), 1/Ta)

# Apenas os valores positivos da FFT e das frequências
positive_frequencies = frequencies[:len(frequencies)//2]
positive_fft = np.abs(tach_signal_fft[:len(tach_signal)//2])

# Criar o gráfico
plt.figure(figsize=(10, 6))
#plt.plot(positive_frequencies, positive_fft, color='blue', label='Espectro de Frequências')
plt.plot(x_percent, normal_distribution, label='Distribuição Normal dos Períodos', color='blue')

# Adicionar marcações para o período médio e desvio padrão (em percentual)
plt.axvline(0, color='red', linestyle='--', label='Período Médio: 0%')  # Média está em 0%
plt.axvline(100, color='green', linestyle='--', label='+1 Desvio Padrão: +100%')  # +1 std = +100%
plt.axvline(-100, color='green', linestyle='--', label='-1 Desvio Padrão: -100%')  # -1 std = -100%


# Personalizar o gráfico
plt.title('Transformada de Fourier do Sinal do Tacômetro')
plt.xlabel('Frequência (Hz)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.legend()

# Exibir o gráfico
plt.show()


- Frequência fundamental da rotação em ZERO.
- Decaimento de forma rápida, o que é típico de sinais que possuem um forte componente de baixa frequência e menos contribuições significativas em frequências mais altas. Define-se então rotação relativamente estável, com poucas vibrações.

In [ ]:
# Leitura em faixa de interesse:
plt.figure(figsize=(10, 6))
plt.plot(positive_frequencies, positive_fft, color='blue', label='Espectro de Frequências')
# Definir o limite de frequência para 1000 Hz
# plt.xlim([0, 1000]) - para limitar a banda de plotagem
# Personalizar o gráfico
plt.title('Transformada de Fourier do Sinal do Tacômetro (Frequências até 500 Hz)')
plt.xlabel('Frequência (Hz)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.legend()
# Exibir o gráfico
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Supondo que tach_signal e Ta estão definidos
# Exemplo: tach_signal = df_10['f_rotac'].values, Ta = 0.00002 s (50 kHz)

# Converter tach_signal para array NumPy
tach_signal = np.array(tach_signal)

# Parâmetros do sinal
Ta = 0.00002  # Tempo de amostragem (50 kHz)
fs = 1 / Ta  # Frequência de amostragem = 50000 Hz
N = len(tach_signal)  # Número de amostras

# Aplicar a Transformada de Fourier no sinal do tacômetro
tach_signal_fft = np.fft.fft(tach_signal)

# Calcular a magnitude da FFT e normalizar
positive_fft = 2 * np.abs(tach_signal_fft[:N//2]) / N  # Magnitude normalizada (fator 2 para compensar frequências negativas)

# Calcular as frequências correspondentes
frequencies = np.fft.fftfreq(N, Ta)
positive_frequencies = frequencies[:N//2]  # Frequências positivas (em Hz)

# Criar o gráfico
plt.figure(figsize=(10, 6))
plt.plot(positive_frequencies, positive_fft, color='blue', label='Espectro de Frequências')

# Definir o limite de frequência para 500 Hz
plt.xlim([0, 600])

# Ajustar o limite do eixo y para melhor visualização
plt.ylim([0, np.max(positive_fft) * 1.1])  # Ajusta o eixo y para mostrar até 10% acima do valor máximo

# Personalizar o gráfico
plt.title('Transformada de Fourier do Sinal do Tacômetro (Frequências até 500 Hz)')
plt.xlabel('Frequência (Hz)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.legend()

# Exibir o gráfico
plt.show()

#### Aplicando filtros:

- Vamos aplicar um filtro para eliminar essas freq. acima de 100Hz, a princípio e verificar a forma de onda do sinal depois.
- aplicar um filtro passa-baixa (FPB) ao sinal no domínio do tempo (tach_signal) antes de calcular a FFT. Filtro passa-baixa Butterworth, que é comum para esse propósito. Depois, recalcularemos a FFT do sinal filtrado e plotaremos o espectro.
- Os valores b e a são arrays NumPy que representam os coeficientes do filtro e são calculados automaticamente pela função com base nos parâmetros fornecidos: a ordem do filtro, a frequência de corte normalizada e o tipo de filtro.
- b: Coeficientes do numerador do filtro (coeficientes da entrada).
- a: Coeficientes do denominador do filtro (coeficientes da saída, para filtros recursivos).
- order = 5: Ordem do filtro (define a suavidade da transição entre a banda de passagem e a banda de rejeição).



In [ ]:
from scipy.signal import butter, lfilter

# Supondo que tach_signal e Ta estão definidos
# Exemplo: tach_signal = df_10['f_rotac'].values, Ta = 0.00002 s (50 kHz)

# Converter tach_signal para array NumPy
tach_signal = np.array(tach_signal)

# Parâmetros do sinal
Ta = 0.00002  # Tempo de amostragem (50 kHz)
fs = 1 / Ta  # Frequência de amostragem = 50000 Hz
N = len(tach_signal)  # Número de amostras

# Definir a frequência de corte do filtro passa-baixa (100 Hz)
cutoff_freq = 100  # Frequência de corte em Hz
order = 5  # Ordem do filtro Butterworth

# Normalizar a frequência de corte em relação à frequência de Nyquist (fs/2)
nyquist_freq = fs / 2
normalized_cutoff = cutoff_freq / nyquist_freq

# Projetar o filtro passa-baixa Butterworth
b, a = butter(order, normalized_cutoff, btype='low', analog=False)

# Aplicar o filtro ao sinal
tach_signal_filtered = lfilter(b, a, tach_signal)

# Calcular a FFT do sinal filtrado
tach_signal_fft = np.fft.fft(tach_signal_filtered)
positive_fft = 2 * np.abs(tach_signal_fft[:N//2]) / N  # Magnitude normalizada

# Calcular as frequências correspondentes
frequencies = np.fft.fftfreq(N, Ta)
positive_frequencies = frequencies[:N//2]  # Frequências positivas (em Hz)

# Criar o gráfico
plt.figure(figsize=(10, 6))
plt.plot(positive_frequencies, positive_fft, color='blue', label='Espectro de Frequências')

# Definir o limite de frequência para 500 Hz
plt.xlim([0, 500])

# Ajustar o limite do eixo y para melhor visualização
plt.ylim([0, np.max(positive_fft) * 1.1])

# Personalizar o gráfico
plt.title('Transformada de Fourier do Sinal do Tacômetro (Após Filtro Passa-Baixa de 100 Hz)')
plt.xlabel('Frequência (Hz)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.legend()

# Exibir o gráfico
plt.show()


In [ ]:
print("Coeficientes b:", b)
print("Coeficientes a:", a)

- Analisando sinal do tacômetro já filtrado:

In [ ]:
tach_signal_filtered = np.array(tach_signal_filtered)

# Criar o eixo de tempo
Ta = 0.00002
time = np.arange(len(tach_signal_filtered)) * Ta  # Eixo do tempo (em segundos)
mask = (time>=1) & (time<= 2)  # Máscara para o intervalo
time_interval = time[mask]  # Tempos no intervalo
tach_signal_interval = tach_signal_filtered[mask]  # Sinal no intervalo
# Filtrar os índices de cruzamentos por zero para o intervalo
zero_crosses_interval = zero_crosses_indices[(zero_crosses_indices >= np.where(mask)[0][0]) & 
                                             (zero_crosses_indices <= np.where(mask)[0][-1])]

# Ajustar os índices para o novo intervalo
zero_crosses_interval = zero_crosses_interval - np.where(mask)[0][0]

# Garantir que os índices de zero_crosses_interval estejam dentro do intervalo válido de time_interval
zero_crosses_interval = zero_crosses_interval[zero_crosses_interval < len(time_interval)]

# Gerar o gráfico
plt.figure(figsize=(14, 6))
plt.plot(time_interval, tach_signal_interval, label='Sinal do Tacômetro')
plt.scatter(time_interval[zero_crosses_interval], tach_signal_interval[zero_crosses_interval], 
            color='red', label='Cruzamentos por Zero', zorder=5)    
plt.title('Sinal do Tacômetro com Cruzamentos por Zero (1 a 2 segundos)')
plt.xlabel('Tempo (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
# Exibir o gráfico
plt.show()


In [ ]:
#Criando uma coluna de tempo para o sinal filtrado:
time_filtered = np.arange(len(tach_signal_filtered)) * Ta  # Eixo do tempo (em segundos)
df_filtered = pd.DataFrame({'time': time, 'filtered_signal': tach_signal_filtered})
df_filtered.head(5)
df_filtered.to_csv('C:/Files/mafaulda_full/imbalance/10g/13.9264_f.csv', index=False)
df_novo = pd.read_csv('C:/Files/mafaulda_full/imbalance/10g/13.9264_f.csv')
df_novo.head(5)
df_novo.tail(5)

In [ ]:
 

plt.figure(figsize=(14, 6))
plt.plot(df_filtered['time'], df_filtered['filtered_signal'], label='Sinal do Tacômetro Filtrado')
plt.scatter(df_filtered['time'][zero_crosses_interval], df_filtered['filtered_signal'][zero_crosses_interval], 
            color='red', label='Cruzamentos por Zero', zorder=1)
plt.title('Sinal do Tacômetro Filtrado com Cruzamentos por Zero (1 a 2 segundos)')  
plt.xlabel('Tempo (s)')
plt.ylabel('Amplitude')
plt.legend()
plt.grid(True)
# Exibir o gráfico
plt.show()





## - Análise para o sinal desbalanceado com peso de 10g em relação à coluna Hunderhang_10g x axial:

In [ ]:
#tac_imbal_10g = '/kaggle/input/mafaulda-full/mafaulda/imbalance/10g/13.9264.csv'
tac_imbal_10g = 'C:/Files/mafaulda_full/imbalance/10g/13.9264.csv'
tac_imbal_10g_f = 'C:/Files/mafaulda_full/imbalance/10g/13.9264_f.csv'  
df = pd.read_csv(tac_imbal_10g_f)
print(df.head(5))

In [ ]:
# Exibir os tipos de dados da coluna 2 (acelerômetro eixo axial) sem excluir dados
tac_imbal_10g = df.iloc[0:, 1]  # Coluna 2 (índice 1)
print("Primeiros registros da Coluna 2 (Acelerômetro Eixo Axial):")
print(tac_imbal_10g.head())

In [ ]:
# Agora que temos a coluna correta, não excluímos dados
accelerometer_axial = df.iloc[1:, 1]  # Coluna 2: acelerômetro eixo axial

In [ ]:
# Função para detectar zero-crossings no sinal do acelerômetro
def zero_crossings(signal):
    zero_crosses = np.where(np.diff(np.sign(signal)))[0]
    return zero_crosses

# Função para estimar a frequência de vibração utilizando zero-crossing
def estimate_vibration_frequency(signal, Ta):
    zero_crosses_indices = zero_crossings(signal)
    if len(zero_crosses_indices) < 2:
        return np.nan  # Evitar calcular se houver menos de 2 cruzamentos por zero
    time_crossings = zero_crosses_indices / Ta  # Converter os índices de cruzamento em tempo
    periods = np.diff(time_crossings)  # Diferença entre os tempos dos cruzamentos
    mean_period = np.mean(periods)
    frequency_of_vibration = 1 / mean_period if mean_period != 0 else 0
    return frequency_of_vibration

In [ ]:
# Calcular a frequência de vibração para o acelerômetro axial
frequency_axial_vibration = estimate_vibration_frequency(accelerometer_axial, Ta)

In [ ]:
# Imprimir a frequência de vibração no eixo axial
if np.isnan(frequency_axial_vibration):
    print("Erro: Não foi possível calcular a frequência devido à falta de cruzamentos suficientes.")
else:
    print("Frequência de Vibração no Eixo Axial com Desbalanceamento de 10g:", frequency_axial_vibration)

- estou convertendo todos os dados da coluna 2 para numéricos, e quaisquer valores inválidos serão convertidos em NaN.
- dropna(): Todos os valores NaN resultantes de conversões inválidas serão removidos, garantindo que o sinal seja composto apenas por números antes de aplicar a detecção de zero-crossings.

In [ ]:
# Coluna 0: Velocidade registrada pelo tacômetro
tacometer_speed = pd.to_numeric(df.iloc[:, 0], errors='coerce').dropna()

# Coluna 1: Acelerômetro axial
accelerometer_axial = pd.to_numeric(df.iloc[:, 1], errors='coerce').dropna()

In [ ]:
# Garantir que ambas as colunas tenham o mesmo tamanho após remover valores inválidos
min_length = min(len(tacometer_speed), len(accelerometer_axial))
tacometer_speed = tacometer_speed.iloc[:min_length]
accelerometer_axial = accelerometer_axial.iloc[:min_length]

# Calcular a correlação de Pearson entre o tacômetro e o acelerômetro axial
correlation = np.corrcoef(tacometer_speed, accelerometer_axial)[0, 1]

# Exibir o resultado da correlação
print("Correlação de Pearson entre a Velocidade do Tacômetro e o Desbalanceamento Detectado no Acelerômetro Axial:", correlation)

- não há uma correlação linear significativa entre a velocidade do tacômetro e o desbalanceamento detectado no acelerômetro axial.